In [1]:
import sqlite3

In [2]:
with sqlite3.connect("scrabble.db") as conn:
    conn.execute("""
        CREATE TABLE players (
            player_id INTEGER PRIMARY KEY,
            name TEXT NOT NULL
        )
    """)

In [3]:
with sqlite3.connect("scrabble.db") as conn:
    conn.execute("""
        INSERT INTO players (name)
        VALUES ('Deniz'), ('Danyel'), ('Robin')
    """)

In [17]:
import pandas as pd

df = pd.read_csv("scrabble_data.csv")

df["winner"].value_counts()

winner
Robin            43
Danyel           36
Deniz            31
Unentschieden     2
Name: count, dtype: int64